In [127]:
import pandas as pd
import numpy as np
import dash
from dash import html, dcc, callback, Input, Output
import plotly.express as px
import dash_mantine_components as dmc

In [128]:
# Load the dataset
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/d51iMGfp_t0QpO30Lym-dw/automobile-sales.csv')

In [129]:
# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

In [130]:
# Initialize the app
app = dash.Dash(__name__)
app.title = 'Automobile Analysis'

In [131]:
# Helper function for layouting the plots
def style_fig(figure):
    figure.update_layout(
        plot_bgcolor = 'rgba(0,0,0,0)',
        paper_bgcolor= 'rgba(0,0,0,0)',
        font_family = "Inter, sans-serif",
        title_font_size = 16,
        title_font_weight = 600,
        title_x = 0.5,
        margin=dict(l=20, r=20, t=40, b=20)
    )
    figure.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#f1f3f5")
    figure.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#f1f3f5")
    
    return figure

In [132]:
# Create the app layout

# Determine spansize for later use for layouting
spansize ={"base":12, "md": 6}

# MantineProvider is the "Wrapper" that allows us to use the theme
app.layout = dmc.MantineProvider(
    theme={
        "fontFamily" : "'Inter', sans-serif",
        "primaryColor": "indigo", # Change this to 'blue', 'orange', 'grape', etc.
        "components": {
            "Button": {"styles" : {"root": {"fontWeight": 400}}},
        },
    },
    children=[
        dmc.Container([
            # --- HEADER ---
            # We use dmc.Paper to create a nice white header bar
            dmc.Paper(
                children=[
                    dmc.Title("Automobile Sales Analysis", order=2, c="indigo", ta="center"),
                    dmc.Text("Explore historical trends and recession impacts", c="dimmed", ta="center", size="sm")
                ],
                shadow = "xs",
                p = "md",
                radius="md",
                withBorder=True,
                mb="lg", # Margin bottom large
                mt="md" # Margin top medium
            ),
            
            # --- CONTROLS SECTION ---
            # Using dmc.Grid to lay out our dropdowns side-by-side
            dmc.Grid([

                # Column 1: Report Type
                dmc.GridCol([
                    dmc.Select(
                        id='dropdown-statistics',
                        label="Select Report Type",
                        description="Choose the analysis perspective",
                        data=[
                            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                        ],
                        value='Recession Period Statistics', # Default value
                        clearable=False,
                        style={"width": "100%"}
                    ) 
                ], span=spansize), # span=6 means "take up half the width" (out of 12)

                # Column 2: Year Selection
                dmc.GridCol([
                    dmc.Select(
                        id='select-year',
                        label="Select Year",
                        description="Filter data by spesific year",
                        data=[{"label": str(i), "value":str(i)} for i in range(1980, 2024, 1)],
                        value=2023, # Default value
                        disabled = True, # Initially disabled
                        style={"width": "100%"}
                    )
                ], span=spansize),
            ], gutter="md", mb="xl"), #gutter adds space between cols

            # --- OUTPUT SECTION ---
            # This is where our graphs will appear
            html.Div(id='output-container')

        ], fluid=True, style={"maxWidth": "1200px"}) # Constrain width for a cleaner look
    ]
)

In [133]:
# Callback for Enabling/Disabling Year Dropdown
@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_input_container(selected_statistics):
    return False if selected_statistics == 'Yearly Statistics' else True

In [134]:
# Main Callback for Plotting
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'),
     Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):
    
    # --- Recession Statistics ---
    if selected_statistics == "Recession Period Statistics":
        recession = data[data['Recession'] == 1]

        # Plot 1: Line chart
        yearly_rec = recession.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(yearly_rec, x='Year', y='Automobile_Sales', markers=True,
                       title="Average Sales Fluctuation (Recession)",
                       labels={'Automobile_Sales': 'Average Automobile Sales'})
        
        # Plot 2: Bar Chart
        avg_sales = recession.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig2 = px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                      title="Average Sales by Vehicle Type",
                      labels={'Vehicle_Type': 'Vehicle Type',
                              'Automobile_Sales': 'Average Automobile Sales'})
        
        # Plot 3: Donut Chart
        exp_rec = recession.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig3 = px.pie(exp_rec, values='Advertising_Expenditure', names='Vehicle_Type',
                      title="Ad Expenditure Share", hole=0.5)
        
        # Plot 4: Bar Chart (Unemployment)
        unemp_data = recession.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        fig4 = px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                      title='Effect of Unemployment Rate',
                      labels={'unemployment_rate': 'Unemployment Rate',
                              'Automobile_Sales': 'Average Automobile Sales'})
        
        # Return the layout
        return dmc.Grid([
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig1)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig2)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig3)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig4)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize)
        ], gutter="md")
    
    # --- Yearly Statistics ---
    elif (input_year and selected_statistics == 'Yearly Statistics'):
        yearly_data = data[data['Year'] == int(input_year)]

        # Plot 1: Trend Line
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        fig1 = px.line(yas, x='Year', y='Automobile_Sales', markers=True,
                       title="Yearly Sales Trend",
                       labels={'Automobile_Sales': 'Average Automobile Sales'})
        
        # Plot 2: Month Sales
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        mas['Month'] = pd.Categorical(mas['Month'], categories=month_order, ordered=True)
        mas = mas.sort_values('Month')
        fig2 = px.line(mas, x='Month', y='Automobile_Sales', markers=True,
                       title=f"Total Monthly Sales in {input_year}",
                       labels={'Automobile_Sales': 'Automobile Sales'})
        
        # Plot 3: Bar Chart
        avg_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        fig3 = px.bar(avg_vdata, x='Vehicle_Type', y='Automobile_Sales', color='Vehicle_Type',
                      title=f"Sales by Type in {input_year}",
                      labels={'Vehicle_Type': 'Vehicle Type',
                              'Automobile_Sales': 'Average Automobile Sales'})
        
        # Plot 4: Pie Chart
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        fig4 = px.pie(exp_data, values='Advertising_Expenditure', names='Vehicle_Type',
                      title=f"Ad Expenditure in {input_year}", hole=0.5)
        
        # return the layout
        return dmc.Grid([
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig1)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig2)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig3)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize),
            dmc.GridCol(dmc.Paper(dcc.Graph(figure=style_fig(fig4)), withBorder=True, shadow="sm", radius="lg", p="md"), span=spansize)
            ], gutter="md")
    
    return None
        

In [135]:
if __name__ == "__main__":
    app.run(debug=True, jupyter_mode='external', port=8053, host='0.0.0.0')

Dash app running on http://0.0.0.0:8053/
